In [1]:
import numpy as np
import pandas as pd

import conv_model as conv
import tensorflow as tf

In [2]:
#### User input ####

## path of the train and test data
train_data = "../data/digit-recognizer/train.csv"
test_data = "../data/digit-recognizer/test.csv"

## path of the test predictions
test_pred_path = "../data/digit-recognizer/test_preds1.csv"

In [3]:
## read data into Dataframe
df_train = pd.read_csv(train_data)
df_train.info()

df_test = pd.read_csv(test_data)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [4]:
## get the first 20000 rows of the data
#df_train = df_train.iloc[:20000]
#print(len(df_train.columns))

In [5]:
## function to create input images
def create_input_images(df):
    im_arr = []
    for row in range(0, len(df.index)):
        ## create the image frame
        image = np.zeros((28, 28, 1))

        if(row%1000 == 0):
            print(row)
        
        ## loop over the pixel values
        for col in range(1, len(df.columns)):
            image[(col-1)//28][(col-1)%28] = np.array([df.iloc[row, col]])
    
        im_arr.append(image)

    return np.array(im_arr)

In [6]:
## create the arrays of input images
im_arr = create_input_images(df_train)
im_arr_test = create_input_images(df_test)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000


In [7]:
conv1 = conv.ConvModel()
model1 = conv1.get_model()

(None, 784)


In [8]:
from tensorflow.keras.losses import CategoricalCrossentropy

In [9]:
## compile the model
model1.compile(optimizer = 'adam', loss = CategoricalCrossentropy(), metrics = ["accuracy"])

## create training input and one_hot encode
Y = df_train["label"].to_numpy()
Y_oh = tf.one_hot(Y, 10)

In [10]:
## fit the model
model1.fit(im_arr, Y_oh, batch_size = 32, epochs = 6)

Epoch 1/6


2024-04-03 15:46:40.452028: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1313/1313 [==============================] - 12s 9ms/step - loss: 0.2904 - accuracy: 0.9097
Epoch 2/6
1313/1313 [==============================] - 12s 9ms/step - loss: 0.1102 - accuracy: 0.9654
Epoch 3/6
1313/1313 [==============================] - 12s 9ms/step - loss: 0.0847 - accuracy: 0.9732
Epoch 4/6
1313/1313 [==============================] - 12s 9ms/step - loss: 0.0687 - accuracy: 0.9788
Epoch 5/6
1313/1313 [==============================] - 12s 9ms/step - loss: 0.0589 - accuracy: 0.9819
Epoch 6/6
1313/1313 [==============================] - 12s 9ms/step - loss: 0.0512 - accuracy: 0.9841


In [11]:
## predict for the test data
preds = model1.predict(im_arr_test)

875/875 [==============================] - 3s 3ms/step


In [19]:
## add test predictions to the test dataframe
#print(preds.shape)
#print(len(np.argmax(preds, axis = 1)))
#print(len(np.arange(1, len(df_test.index)+1)))
#print(df_test.columns)

df_test["Label"] = np.argmax(preds, axis = 1)
df_test["ImageId"] = np.arange(1, len(df_test.index)+1)

## write results to csv file
to_write = ["ImageId", "Label"]
df_test.to_csv(test_pred_path, columns = to_write, index = False)